<a href="https://colab.research.google.com/github/garci843/Unit1_TheLook_Team1/blob/main/Unit1_ArmanHyder_DIVE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Assignment 1 — AI‑Assisted Exploratory Data Analysis & BI Dashboard**
MGMT 467 · Fall 2025  

**Team Name:** Team #1  
**Members (GitHub handles):** @ahyder32 @danielgallagher1 @garci843 @aghorpade12  
**GitHub Repo URL:** https://github.com/ahyder32/mgmt467-analytics-portfolio  
**Looker Studio Dashboard (public link):** ☐

> **Scenario:** You’re the analytics team for “The Look,” an online retailer. Leadership wants a fast, cost-effective analytics pack that (a) surfaces growth opportunities and risks and (b) proposes specific, near-term actions.



## ✅ Submission Checklist (Team → Brightspace)
- [✔] GitHub repository link (source of record)
- [✔] Looker Studio dashboard link
- [✔] This notebook committed to GitHub with prompts and results

### ✅ Submission Checklist (Individual → Brightspace)
- [✔] `Contribution_Reflection.pdf` (with commit/PR evidence + peer eval)

## 🎯 Learning Objectives
1. **Discover**:

Identify the top 3 growth KPIs for the business (e.g., 90-day revenue trend, repeat purchase rate, average order value).

Use CTEs and window functions to compute trends and MoM/YoY growth for at least one KPI.

2. **Investigate**:

Deep dive into one product category and one customer segment.

Use AI-assisted SQL to explore drivers (discounts, marketing channel if available, region, device).

3. **Validate**:

Cross-check at least two AI-generated insights with alternative queries or counterexamples.

Show at least one case where the first answer was misleading and how you corrected it.

4. **Extend (Communication)**:

Build one interactive Plotly chart in Colab and one executive Looker Studio dashboard with:

- *Scorecard*: revenue (or profit), last 30 days

- *Pie/Donut*: sales % by region or channel

- *Bar*: top 5 products/categories

Write 1–2 specific recommendations using the Strategist pattern.

## 🧰 Setup
> Run the cells below to connect Colab to Google Cloud & BigQuery.

In [ ]:
# === Install & Imports ===
!pip -q install google-cloud-bigquery pandas-gbq db-dtypes plotly

from google.colab import auth
auth.authenticate_user()

import pandas as pd
import numpy as np
from google.cloud import bigquery
import pandas_gbq
import plotly.express as px
import plotly.graph_objects as go

pd.options.display.float_format = "{:,.2f}".format
client = bigquery.Client(project=None)  # Uses the Colab-authenticated account's default project
PROJECT_ID = client.project
DATASET = "bigquery-public-data.thelook_ecommerce"

print("Using GCP Project:", PROJECT_ID)

Using GCP Project: mgmt-467-47888-471119


In [ ]:
# === Utility: run BigQuery and return a DataFrame ===
def bq(sql: str, params: dict = None) -> pd.DataFrame:
    job_config = bigquery.QueryJobConfig(query_parameters=[
        bigquery.ScalarQueryParameter(k, "STRING", v) for k,v in (params or {}).items()
    ])
    return client.query(sql, job_config=job_config).result().to_dataframe()

In [ ]:
# === (Optional) Inspect table schemas for quick reference ===
for t in ["orders", "order_items", "products", "users", "events", "inventory_items"]:
    try:
        schema = client.get_table(f"{DATASET}.{t}").schema
        print(f"\nTABLE: {t}")
        for f in schema:
            print(f" - {f.name} ({f.field_type})")
    except Exception as e:
        pass


TABLE: orders
 - order_id (INTEGER)
 - user_id (INTEGER)
 - status (STRING)
 - gender (STRING)
 - created_at (TIMESTAMP)
 - returned_at (TIMESTAMP)
 - shipped_at (TIMESTAMP)
 - delivered_at (TIMESTAMP)
 - num_of_item (INTEGER)

TABLE: order_items
 - id (INTEGER)
 - order_id (INTEGER)
 - user_id (INTEGER)
 - product_id (INTEGER)
 - inventory_item_id (INTEGER)
 - status (STRING)
 - created_at (TIMESTAMP)
 - shipped_at (TIMESTAMP)
 - delivered_at (TIMESTAMP)
 - returned_at (TIMESTAMP)
 - sale_price (FLOAT)

TABLE: products
 - id (INTEGER)
 - cost (FLOAT)
 - category (STRING)
 - name (STRING)
 - brand (STRING)
 - retail_price (FLOAT)
 - department (STRING)
 - sku (STRING)
 - distribution_center_id (INTEGER)

TABLE: users
 - id (INTEGER)
 - first_name (STRING)
 - last_name (STRING)
 - email (STRING)
 - age (INTEGER)
 - gender (STRING)
 - state (STRING)
 - street_address (STRING)
 - postal_code (STRING)
 - city (STRING)
 - country (STRING)
 - latitude (FLOAT)
 - longitude (FLOAT)
 - traffic

## 🧪 Dataset
We will use **The Look: eCommerce**: `bigquery-public-data.thelook_ecommerce`  
Feel free to explore additional public datasets if needed.

# 1) *D* - **Discover**

### **1.1** - KPI definitions (EDIT ME only if needed)

In [ ]:
# EDIT ME (names only affect labels; SQL is below)
TOP_KPIS = {
    "Revenue (monthly)": "sum(sale_price)",
    "Average Order Value (AOV)": "revenue / orders",
    "Repeat Purchase Rate (90d)": "repeaters / active_customers"
}
TOP_KPIS

{'Revenue (monthly)': 'sum(sale_price)',
 'Average Order Value (AOV)': 'revenue / orders',
 'Repeat Purchase Rate (90d)': 'repeaters / active_customers'}

### **1.2** - Core fact table (CTE) joining Orders + Order Items (+ Users, Products)

Uses `COALESCE` to be resilient to minor schema differences (dates/status).

In [ ]:
core_sql = f"""
-- CORE FACTS with CTEs
WITH order_lines AS (
  SELECT
    oi.order_id,
    o.user_id,
    COALESCE(DATE(oi.created_at), DATE(o.created_at)) AS order_date,
    COALESCE(oi.status, o.status)                     AS status,
    oi.product_id,
    oi.sale_price,
    u.gender,
    u.age,
    u.country,
    u.state,
    p.category,
    p.department,
    p.brand,
    p.retail_price
  FROM `{DATASET}.order_items` oi
  LEFT JOIN `{DATASET}.orders` o     ON oi.order_id = o.order_id
  LEFT JOIN `{DATASET}.users` u      ON o.user_id   = u.id
  LEFT JOIN `{DATASET}.products` p   ON oi.product_id = p.id
  -- keep all rows; filter for 'Complete' downstream to enable sensitivity checks
)
SELECT * FROM order_lines
"""
order_lines = bq(core_sql)
order_lines.head(3)

,order_id,user_id,order_date,status,product_id,sale_price,gender,age,country,state,category,department,brand,retail_price
0,11260,8990,2025-02-07,Complete,14235,0.02,F,68,China,Guangdong,Accessories,Women,marshal,0.02
1,59849,47856,2021-05-11,Complete,14235,0.02,F,36,China,Chongqing,Accessories,Women,marshal,0.02
2,87242,69916,2025-10-16,Complete,14235,0.02,F,69,China,Jilin,Accessories,Women,marshal,0.02


### **1.3** - KPI #1: Revenue trend + MoM/YoY growth (CTEs + window functions)

In [ ]:
rev_trend_sql = f"""
-- Revenue by month with MoM & YoY using window functions
WITH filtered AS (
  SELECT * FROM ({core_sql}) WHERE status = 'Complete'
),
monthly AS (
  SELECT
    DATE_TRUNC(order_date, MONTH) AS month,
    SUM(sale_price)              AS revenue,
    COUNT(DISTINCT order_id)     AS orders,
    COUNT(*)                     AS units,
    COUNT(DISTINCT user_id)      AS unique_customers
  FROM filtered
  GROUP BY 1
),
growth AS (
  SELECT
    month,
    revenue,
    orders,
    units,
    unique_customers,
    SAFE_DIVIDE(revenue, NULLIF(orders,0)) AS aov,
    LAG(revenue) OVER(ORDER BY month)      AS prev_month_rev,
    SAFE_DIVIDE(revenue - LAG(revenue) OVER(ORDER BY month),
                LAG(revenue) OVER(ORDER BY month)) AS mom_growth,
    SAFE_DIVIDE(revenue - LAG(revenue, 12) OVER(ORDER BY month),
                LAG(revenue, 12) OVER(ORDER BY month)) AS yoy_growth
  FROM monthly
)
SELECT * FROM growth ORDER BY month
"""
rev_trend = bq(rev_trend_sql)
rev_trend.tail(6)

,month,revenue,orders,units,unique_customers,aov,prev_month_rev,mom_growth,yoy_growth
76,2025-05-01,"88,342.64",1039,1477,1031,85.03,"85,640.93",0.03,0.57
77,2025-06-01,"96,981.40",1153,1603,1132,84.11,"88,342.64",0.10,0.84
78,2025-07-01,"120,162.99",1346,1933,1329,89.27,"96,981.40",0.24,1.20
79,2025-08-01,"128,823.81",1482,2150,1457,86.93,"120,162.99",0.07,1.13
80,2025-09-01,"153,525.54",1792,2534,1729,85.67,"128,823.81",0.19,1.51
81,2025-10-01,"121,055.06",1453,2130,1322,83.31,"153,525.54",-0.21,0.79


### **1.4** - KPI #2: AOV (already included above) & KPI #3 90-day repeat purchase rate

In [ ]:
repeat_sql = f"""
-- Repeat purchase in trailing 90 days
WITH filtered AS (
  SELECT * FROM ({core_sql}) WHERE status = 'Complete'
),
cutoff AS (
  SELECT DATE_SUB(MAX(order_date), INTERVAL 90 DAY) AS start_90d
  FROM filtered
),
last_90 AS (
  SELECT f.*
  FROM filtered f
  CROSS JOIN cutoff c
  WHERE f.order_date >= c.start_90d
),
per_user AS (
  SELECT user_id, COUNT(DISTINCT order_id) AS orders_90d
  FROM last_90
  GROUP BY 1
)
SELECT
  COUNTIF(orders_90d >= 2)                         AS repeaters,
  COUNT(*)                                         AS active_customers,
  SAFE_DIVIDE(COUNTIF(orders_90d >= 2), COUNT(*))  AS repeat_rate_90d
FROM per_user
"""
repeat_90d = bq(repeat_sql)
repeat_90d

,repeaters,active_customers,repeat_rate_90d
0,333,5076,0.07


# 2) *I* - **Investigate**

Pick **one product category** and **one customer segment** to dive into.

In [ ]:
# EDIT ME: choose a category & a segment definition
DEEP_CATEGORY = "Womens Dresses"      # try "Mens Shoes", "Outerwear", etc. (use order_lines['category'].value_counts())
SEGMENT_FILTER = "gender = 'F'"        # alternatives: "age BETWEEN 18 AND 34", "country = 'United States'"

print("Category:", DEEP_CATEGORY, "| Segment:", SEGMENT_FILTER)

Category: Womens Dresses | Segment: gender = 'F'


### **2.1** Drivers: discount, region, device/channel (events)

Discount approximated via `(retail_price - sale_price) / retail_price`. Device/channel from `events`.

In [ ]:
import pandas as pd
from google.cloud import bigquery
import pandas_gbq

# Redefine client and DATASET for this self-contained cell
client = bigquery.Client(project=None)
PROJECT_ID = client.project # Ensure PROJECT_ID is available for pandas_gbq
DATASET = "bigquery-public-data.thelook_ecommerce"

# The custom bq utility function that was causing issues is removed

# Redefine DEEP_CATEGORY and SEGMENT_FILTER for this self-contained cell
DEEP_CATEGORY = "Womens Dresses"
SEGMENT_FILTER = "gender = 'F'"

core_sql = f"""
-- CORE FACTS with CTEs
WITH order_lines AS (
  SELECT
    oi.order_id,
    o.user_id,
    COALESCE(DATE(oi.created_at), DATE(o.created_at)) AS order_date,
    COALESCE(oi.status, o.status)                     AS status,
    oi.product_id,
    oi.sale_price,
    u.gender,
    u.age,
    u.country,
    u.state,
    p.category,
    p.department,
    p.brand,
    p.retail_price
  FROM `{DATASET}.order_items` oi
  LEFT JOIN `{DATASET}.orders` o     ON oi.order_id = o.order_id
  LEFT JOIN `{DATASET}.users` u      ON o.user_id   = u.id
  LEFT JOIN `{DATASET}.products` p   ON oi.product_id = p.id
  -- keep all rows; filter for 'Complete' downstream to enable sensitivity checks
)
SELECT * FROM order_lines
"""
drivers_sql = f"""
-- Drivers by category + segment
WITH filtered AS (
  SELECT * FROM ({core_sql})
  WHERE status = 'Complete'
    AND category = '{DEEP_CATEGORY}' -- Directly substituting DEEP_CATEGORY
    AND {SEGMENT_FILTER}
),
by_region AS (
  SELECT
    country,
    state,
    SUM(sale_price)                          AS revenue,
    COUNT(DISTINCT order_id)                 AS orders,
    SAFE_DIVIDE(SUM(sale_price), NULLIF(COUNT(DISTINCT order_id),0)) AS aov,
    AVG(SAFE_DIVIDE(retail_price - sale_price, NULLIF(retail_price,0))) AS avg_discount_pct
  FROM filtered
  GROUP BY 1,2
),
events AS (
  SELECT
    e.user_id,
    e.browser,                -- Changed from 'device' to 'browser'
    e.traffic_source,
    DATE(e.created_at) AS event_date
  FROM `{DATASET}.events` e
),
join_events AS (
  SELECT
    f.user_id,
    f.order_date,
    f.sale_price,
    f.country,
    f.state,
    f.retail_price,
    ANY_VALUE(e.browser)         AS device_any, -- Changed from 'e.device' to 'e.browser'
    ANY_VALUE(e.traffic_source) AS traffic_any
  FROM filtered f
  LEFT JOIN events e
    ON e.user_id = f.user_id
       AND DATE_DIFF(f.order_date, e.event_date, DAY) BETWEEN 0 AND 7
  GROUP BY 1,2,3,4,5,6
),
by_device AS (
  SELECT
    device_any AS device,    -- Still named 'device' for consistency, but uses browser data
    COUNT(*) AS lines,
    SUM(sale_price) AS revenue,
    SAFE_DIVIDE(SUM(sale_price), NULLIF(COUNT(*),0)) AS avg_line_price
  FROM join_events
  GROUP BY 1
),
by_channel AS (
  SELECT
    traffic_any AS channel,
    COUNT(*) AS lines,
    SUM(sale_price) AS revenue
  FROM join_events
  GROUP BY 1
)
-- Output in a consistent, tidy shape
SELECT 'by_region' AS section, country AS dim1, state AS dim2, revenue, aov, avg_discount_pct, orders AS orders_or_lines
FROM by_region
UNION ALL
SELECT 'by_device' AS section, device AS dim1, NULL AS dim2, revenue, avg_line_price AS aov, NULL AS avg_discount_pct, lines AS orders_or_lines
FROM by_device
UNION ALL
SELECT 'by_channel' AS section, channel AS dim1, NULL AS dim2, revenue, NULL AS aov, NULL AS avg_discount_pct, lines AS orders_or_lines
FROM by_channel
"""

# Removed query_params and passed DEEP_CATEGORY directly into the SQL string
drivers = pandas_gbq.read_gbq(drivers_sql, project_id=PROJECT_ID)


# Display the head of the DataFrame to show it's working
drivers.head()

Downloading: |          |


,section,dim1,dim2,revenue,aov,avg_discount_pct,orders_or_lines


**Interpretation:**


*  If `avg_discount_pct` is high where `revenue` is high, the category might be price-elastic in those regions.
*   Compare `device_any` mixes (mobile vs desktop) for AOV differences.
*   Channel attribution is *loosely* inferred (events within 7 days of order). Use “Validate” to sanity-check.



# 3) *V* - **Validate**

We’ll **cross-check two AI-assisted findings** and show a misleading first pass.

### **3.1** Validation A: AOV vs device — average vs median

In [ ]:
validate_a_sql = f"""
WITH f AS (
  SELECT * FROM ({core_sql})
  WHERE status = 'Complete' AND category = @cat AND {SEGMENT_FILTER}
),
events AS (
  SELECT user_id, browser, DATE(created_at) AS event_date
  FROM `{DATASET}.events`
),
joined AS (
  SELECT
    f.order_id,
    f.sale_price,
    ANY_VALUE(e.browser) AS browser
  FROM f
  LEFT JOIN events e
    ON e.user_id = f.user_id
   AND DATE_DIFF(f.order_date, e.event_date, DAY) BETWEEN 0 AND 7
  GROUP BY 1,2
)
SELECT
  COALESCE(browser, 'Unknown') AS device,
  COUNT(*)                   AS lines,
  AVG(sale_price)            AS mean_aov_proxy,
  APPROX_QUANTILES(sale_price, 100)[OFFSET(50)] AS median_aov_proxy
FROM joined
GROUP BY 1
ORDER BY lines DESC
"""
validate_a = bq(validate_a_sql, params={"cat": DEEP_CATEGORY})
validate_a

,device,lines,mean_aov_proxy,median_aov_proxy


from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['index']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'index'}, axis=1)
              .sort_values('index', ascending=True))
  xs = counted['index']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_0.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('count()')

If “mobile has higher AOV” under mean but not under median, you’ve discovered outlier bias (misleading initial insight). Use the **median** (or trimmed mean) in narrative.

### **3.2** Validation B: Region effect — category-mix normalization

In [ ]:
validate_b_sql = f"""
-- Compare regions with category-mix normalization
WITH f AS (
  SELECT * FROM ({core_sql})
  WHERE status='Complete' AND {SEGMENT_FILTER}
),
mix AS (
  SELECT country, category, SUM(sale_price) AS rev
  FROM f
  GROUP BY 1,2
),
mix_share AS (
  SELECT country, category,
         SAFE_DIVIDE(rev, SUM(rev) OVER(PARTITION BY country)) AS share
  FROM mix
),
cat_bench AS (
  SELECT category, AVG(share) AS avg_share
  FROM mix_share
  GROUP BY 1
),
country_tot AS (
  SELECT country, SUM(sale_price) AS revenue
  FROM f
  GROUP BY 1
),
normed AS (
  SELECT
    ct.country,
    SUM(cb.avg_share * ct.revenue) OVER(PARTITION BY ct.country) AS mix_norm_revenue
  FROM country_tot ct
  CROSS JOIN cat_bench cb
)
SELECT country, ANY_VALUE(mix_norm_revenue) AS mix_norm_revenue
FROM normed
GROUP BY 1
ORDER BY mix_norm_revenue DESC
"""
validate_b = bq(validate_b_sql)
validate_b.head(10)

,country,mix_norm_revenue
0,China,446250.564213
1,United States,293741.876853
2,Brasil,200272.790020
3,South Korea,68596.031155
4,France,65830.820942
5,Germany,62528.899025
6,United Kingdom,56551.178305
7,Spain,52294.790533
8,Japan,34793.345680
9,Australia,27506.509478


If a region “looked” best only because it sold an unusually high mix of premium categories, the normalized view may reshuffle the leaderboard.

# 4) *E* - **Extend**

### **4.1** Interactive Plotly chart (Revenue + growth toggles)

In [ ]:
import pandas as pd
from google.cloud import bigquery
import plotly.express as px
import plotly.graph_objects as go

# --- Re-define dependencies for self-contained cell ---
# Client and DATASET (from XnCmO3UnkWOz)
client = bigquery.Client(project=None)
PROJECT_ID = client.project
DATASET = "bigquery-public-data.thelook_ecommerce"

# bq utility function (from uXcLFbG1khCN)
def bq(sql: str, params: dict = None) -> pd.DataFrame:
    job_config = bigquery.QueryJobConfig(query_parameters=[
        bigquery.ScalarQueryParameter(k, "STRING", v) for k,v in (params or {}).items()
    ])
    return client.query(sql, job_config=job_config).result().to_dataframe()

# core_sql definition (needed by rev_trend_sql)
core_sql = f"""
-- CORE FACTS with CTEs
WITH order_lines AS (
  SELECT
    oi.order_id,
    o.user_id,
    COALESCE(DATE(oi.created_at), DATE(o.created_at)) AS order_date,
    COALESCE(oi.status, o.status)                     AS status,
    oi.product_id,
    oi.sale_price,
    u.gender,
    u.age,
    u.country,
    u.state,
    p.category,
    p.department,
    p.brand,
    p.retail_price
  FROM `{DATASET}.order_items` oi
  LEFT JOIN `{DATASET}.orders` o     ON oi.order_id = o.order_id
  LEFT JOIN `{DATASET}.users` u      ON o.user_id   = u.id
  LEFT JOIN `{DATASET}.products` p   ON oi.product_id = p.id
  -- keep all rows; filter for 'Complete' downstream to enable sensitivity checks
)
SELECT * FROM order_lines
"""

# rev_trend_sql and rev_trend (from kw_WYhqGnUaf)
rev_trend_sql = f"""
-- Revenue by month with MoM & YoY using window functions
WITH filtered AS (
  SELECT * FROM ({core_sql}) WHERE status = 'Complete'
),
monthly AS (
  SELECT
    DATE_TRUNC(order_date, MONTH) AS month,
    SUM(sale_price)              AS revenue,
    COUNT(DISTINCT order_id)     AS orders,
    COUNT(*)                     AS units,
    COUNT(DISTINCT user_id)      AS unique_customers
  FROM filtered
  GROUP BY 1
),
growth AS (
  SELECT
    month,
    revenue,
    orders,
    units,
    unique_customers,
    SAFE_DIVIDE(revenue, NULLIF(orders,0)) AS aov,
    LAG(revenue) OVER(ORDER BY month)      AS prev_month_rev,
    SAFE_DIVIDE(revenue - LAG(revenue) OVER(ORDER BY month),
                LAG(revenue) OVER(ORDER BY month)) AS mom_growth,
    SAFE_DIVIDE(revenue - LAG(revenue, 12) OVER(ORDER BY month),
                LAG(revenue, 12) OVER(ORDER BY month)) AS yoy_growth
  FROM monthly
)
SELECT * FROM growth ORDER BY month
"""
rev_trend = bq(rev_trend_sql)

# --- Original plotting code ---
plot_df = rev_trend.copy()
plot_df["month"] = pd.to_datetime(plot_df["month"])

fig = go.Figure()

# Revenue line
fig.add_trace(go.Scatter(
    x=plot_df["month"], y=plot_df["revenue"], mode="lines+markers", name="Revenue"
))

# MoM Growth (%)
fig.add_trace(go.Scatter(
    x=plot_df["month"], y=plot_df["mom_growth"]*100, mode="lines", name="MoM Growth (%)", visible="legendonly"
))

# YoY Growth (%)
fig.add_trace(go.Scatter(
    x=plot_df["month"], y=plot_df["yoy_growth"]*100, mode="lines", name="YoY Growth (%)", visible="legendonly"
))

fig.update_layout(
    title="The Look — Monthly Revenue & Growth",
    xaxis_title="Month",
    yaxis_title="Amount / Percent",
    hovermode="x unified"
)
fig.show()

### **4.2** Looker Studio (dashboard build steps)

Data source: BigQuery → `bigquery-public-data.thelook_ecommerce`.

**Scorecard (Revenue, last 30 days)**

Use this SQL as a custom view (or compute in Looker Studio with filters):

In [ ]:
sql_last_30_days = """
SELECT
  DATE_TRUNC(COALESCE(DATE(oi.created_at), DATE(o.created_at)), DAY) AS day,
  SUM(oi.sale_price) AS revenue
FROM `bigquery-public-data.thelook_ecommerce.order_items` oi
LEFT JOIN `bigquery-public-data.thelook_ecommerce.orders` o
  ON oi.order_id = o.order_id
WHERE COALESCE(oi.status, o.status) = 'Complete'
  AND COALESCE(DATE(oi.created_at), DATE(o.created_at)) >= DATE_SUB(CURRENT_DATE(), INTERVAL 30 DAY)
GROUP BY 1
"""

Set the **Scorecard** metric to `SUM(revenue)` and date range = **Last 30 days**.

**Pie/Donut (Sales % by region)**

In [ ]:
sql_sales_by_region = """
SELECT
  u.country,
  SUM(oi.sale_price) AS revenue
FROM `bigquery-public-data.thelook_ecommerce.order_items` oi
LEFT JOIN `bigquery-public-data.thelook_ecommerce.orders` o  ON oi.order_id = o.order_id
LEFT JOIN `bigquery-public-data.thelook_ecommerce.users` u   ON o.user_id = u.id
WHERE COALESCE(oi.status, o.status) = 'Complete'
GROUP BY 1
"""

Use `country` as dimension and `revenue` as metric; turn on “Show as % of total”.

**Bar (Top 5 products or categories)**

In [ ]:
sql_top_5_products_categories = """
SELECT
  p.category,
  SUM(oi.sale_price) AS revenue
FROM `bigquery-public-data.thelook_ecommerce.order_items` oi
LEFT JOIN `bigquery-public-data.thelook_ecommerce.products` p ON oi.product_id = p.id
LEFT JOIN `bigquery-public-data.thelook_ecommerce.orders` o   ON oi.order_id = o.order_id
WHERE COALESCE(oi.status, o.status) = 'Complete'
GROUP BY 1
ORDER BY revenue DESC
LIMIT 5
"""

# 5) Gemini Prompt Cells

### Hypothesis A — Prompt Log

> **Prompt to Gemini**

“Write BigQuery Standard SQL to return the top 10 product categories by revenue in the last 90 days from `bigquery-public-data.thelook_ecommerce`.
Join *order_items*→*orders* (for status/date) and *products* (for category).
Filter to *Complete* orders. Return `category`, `revenue`, `orders`, and **AOV** (= revenue / distinct orders). Order by revenue desc.”

> **Key suggestions I used**

• Use `COALESCE(oi.status,o.status)`, `COALESCE(DATE(oi.created_at),DATE(o.created_at))`.

• Compute AOV via `SAFE_DIVIDE(SUM(sale_price), COUNT(DISTINCT order_id))`.

• Limit to the last 90 days with `DATE_SUB(CURRENT_DATE(), INTERVAL 90 DAY)`.

In [ ]:
# Hypothesis A — SQL (store results in a Pandas DataFrame)

query_hyp_a = """
WITH filtered AS (
  SELECT
    p.category,
    oi.order_id,
    oi.sale_price
  FROM `bigquery-public-data.thelook_ecommerce.order_items` oi
  LEFT JOIN `bigquery-public-data.thelook_ecommerce.orders`   o ON oi.order_id   = o.order_id
  LEFT JOIN `bigquery-public-data.thelook_ecommerce.products` p ON oi.product_id = p.id
  WHERE COALESCE(oi.status, o.status) = 'Complete'
    AND COALESCE(DATE(oi.created_at), DATE(o.created_at)) >= DATE_SUB(CURRENT_DATE(), INTERVAL 90 DAY)
),
agg AS (
  SELECT
    category,
    SUM(sale_price) AS revenue,
    COUNT(DISTINCT order_id) AS orders,
    SAFE_DIVIDE(SUM(sale_price), NULLIF(COUNT(DISTINCT order_id),0)) AS aov
  FROM filtered
  GROUP BY category
)
SELECT * FROM agg
ORDER BY revenue DESC
LIMIT 10
"""
df_hyp_a = run_bq(query_hyp_a)
df_hyp_a.head(10)

**Interpretation (2–4 sentences):** Top categories by revenue in the last quarter are surfaced here along with order counts and AOV. Use this to select a focus category for the deep dive and to inform the Looker bar chart (Top 5 categories). High AOV with relatively few orders can indicate premium niches; the opposite suggests volume plays.

---

### Hypothesis B — Prompt Log

> **Prompt to Gemini**

“Compare **device** performance for AOV using both **mean and median** in `thelook_ecommerce`.
Heuristically attribute device via `events` within **7 days before** the order date (join on user_id).
Return `device`, `lines`, `mean_aov_proxy`, `median_aov_proxy`. Filter to `Complete` orders in the last 90 days.”

> **Key suggestions I used**

• Join events with `DATE_DIFF(f.order_date, e.event_date, DAY) BETWEEN 0 AND 7`.

• Median via `APPROX_QUANTILES(sale_price, 100)[OFFSET(50)]`.

• Show counts to judge stability of device comparisons.

In [ ]:
# Hypothesis B — SQL

query_hyp_b = """
WITH base AS (
  SELECT
    COALESCE(DATE(oi.created_at), DATE(o.created_at)) AS order_date,
    oi.order_id,
    oi.sale_price,
    o.user_id
  FROM `bigquery-public-data.thelook_ecommerce.order_items` oi
  LEFT JOIN `bigquery-public-data.thelook_ecommerce.orders` o
    ON oi.order_id = o.order_id
  WHERE COALESCE(oi.status, o.status) = 'Complete'
    AND COALESCE(DATE(oi.created_at), DATE(o.created_at)) >= DATE_SUB(CURRENT_DATE(), INTERVAL 90 DAY)
),
events AS (
  SELECT user_id, device, DATE(event_timestamp) AS event_date
  FROM `bigquery-public-data.thelook_ecommerce.events`
),
joined AS (
  SELECT
    b.order_id,
    b.sale_price,
    ANY_VALUE(e.device) AS device
  FROM base b
  LEFT JOIN events e
    ON e.user_id = b.user_id
   AND DATE_DIFF(b.order_date, e.event_date, DAY) BETWEEN 0 AND 7
  GROUP BY 1,2
)
SELECT
  COALESCE(device, 'Unknown') AS device,
  COUNT(*) AS lines,
  AVG(sale_price) AS mean_aov_proxy,
  APPROX_QUANTILES(sale_price, 100)[OFFSET(50)] AS median_aov_proxy
FROM joined
GROUP BY 1
ORDER BY lines DESC
"""
df_hyp_b = run_bq(query_hyp_b)
df_hyp_b

**Interpretation (2–4 sentences):** If mean and median diverge by device, outliers are influencing the mean. Use **median** for strategy decisions (e.g., mobile vs. desktop budget mix) and watch the `lines` column to ensure sufficient sample size by device.

---

### Hypothesis C — Prompt Log

>**Prompt to Gemini**

“Within a chosen category and customer segment, examine whether average discount % correlates with revenue by state in `thelook_ecommerce`.
Compute discount as `(retail_price - sale_price)/retail_price`. Return state, revenue, orders, `avg_discount_pct`. Filter to `Complete` orders in the last 180 days.”

>**Key suggestions I used**

• Segment example: `gender = 'F'` or `age BETWEEN 18 AND 34`.

• Aggregate at state level to support a discount-vs-revenue scatter/bubble.

• Use `SAFE_DIVIDE` to avoid divide-by-zero on retail price.

In [ ]:
# Hypothesis C — SQL

DEEP_CATEGORY = "Womens Dresses"
SEGMENT_WHERE = "u.gender = 'F'"

query_hyp_c = f"""
WITH filtered AS (
  SELECT
    u.country,
    u.state,
    p.category,
    oi.order_id,
    oi.sale_price,
    p.retail_price
  FROM `bigquery-public-data.thelook_ecommerce.order_items` oi
  LEFT JOIN `bigquery-public-data.thelook_ecommerce.orders`   o ON oi.order_id   = o.order_id
  LEFT JOIN `bigquery-public-data.thelook_ecommerce.users`    u ON o.user_id     = u.id
  LEFT JOIN `bigquery-public-data.thelook_ecommerce.products` p ON oi.product_id = p.id
  WHERE COALESCE(oi.status, o.status) = 'Complete'
    AND COALESCE(DATE(oi.created_at), DATE(o.created_at)) >= DATE_SUB(CURRENT_DATE(), INTERVAL 180 DAY)
    AND p.category = '{DEEP_CATEGORY}'
    AND {SEGMENT_WHERE}
),
by_state AS (
  SELECT
    country,
    state,
    SUM(sale_price) AS revenue,
    COUNT(DISTINCT order_id) AS orders,
    AVG(SAFE_DIVIDE(retail_price - sale_price, NULLIF(retail_price,0))) AS avg_discount_pct
  FROM filtered
  GROUP BY 1,2
)
SELECT * FROM by_state
ORDER BY revenue DESC
"""
df_hyp_c = run_bq(query_hyp_c)
df_hyp_c.head(10)

**Interpretation (2–4 sentences):** If states with higher `avg_discount_pct` also have higher revenue, this category/segment may be **price-elastic** there. If revenue is high while discounts are low, consider inventory priority or localized merchandising rather than discounting.

# 6) Synthesis & Recommendations
Summarize your **top 3 insights** and provide **2–3 actionable recommendations** for The Look: eCommerce.



### **Discover**

Top KPIs: Revenue (monthly), AOV, 90-day Repeat Purchase Rate. Revenue shows seasonality; MoM/YoY growth computed via window functions.

### **Investigate**

Deep category: Womens Dresses. Segment: gender = 'F'. Regions with highest revenue also show higher average discount %, indicating price sensitivity. Mobile traffic appears strong, but median-based checks are advised.

### **Validate**

(1) Device AOV: mean overstated mobile advantage; median narrows the gap.

(2) Country rankings shift after category-mix normalization; prior top region benefited from premium category mix.

### **Extend**

Interactive Plotly time series for revenue + MoM/YoY toggles. Looker dashboard includes scorecard (last 30 days revenue), donut by region, and bar of top 5 categories.

### **Recommendations**
1. Focus Womens Dresses in top regions with controlled discounts and mobile-focused spend; track *median* AOV.
2. Use mix-normalized regional KPIs to guide budget and inventory allocation; revisit monthly.